In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as ss

In [2]:
from statsmodels.formula.api import mixedlm

In [3]:
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', None)

In [4]:
# #get rid of scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [5]:
data = pd.read_csv("data_no_wof_latency_backfilled_happiness.csv")


In [6]:
data = data[~data.Subject.isin([1991118.000, 1991119.000,1991143.000, 1991165.000, 1991117.000])]
data = data[~data.Subject.isin([1991102.000, 1991107.000,1991113.000, 1991117.000, 1991119.000, 1991142, 1991175])] # did not follow instructions



In [7]:
# calculate fatigue
data["tap_count"] = [1] * len(data)
tap_count = data.groupby(['Subject', 'trial_label'])['tap_count'].cumsum().reset_index(name = "fatigue")
tap_count = tap_count.reset_index().set_index("index")
tap_count = tap_count.drop("level_0", axis = 1)


In [8]:
data = tap_count.merge(data, left_index = True, right_index = True, how = "inner")


In [9]:
data["log_latency"] = np.log(data["latency"])


In [10]:
data.price_displayed.unique()


array([0.4, 1. , 0.1, 2.5, nan])

In [11]:
data.task.unique()

array(['fishing_pos', 'mood_rating_pos'], dtype=object)

In [12]:
# drop mood rating trials
data = data[data.task == 'fishing_pos']

In [13]:
data.latency.isnull().values.any()
data = data[data.latency.notnull()]

In [16]:
f_model = mixedlm(formula = "log_latency ~ 1 + response + price_displayed + fatigue + environment",
                 groups = data["Subject"],
                 re_formula = "~ response + price_displayed + fatigue + environment", data = data)
r = f_model.fit(reml = False, method = "powell")


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [17]:
print (r.summary())

                         Mixed Linear Model Regression Results
Model:                       MixedLM           Dependent Variable:           log_latency
No. Observations:            24971             Method:                       ML         
No. Groups:                  19                Scale:                        0.0702     
Min. group size:             978               Likelihood:                   -2405.7414 
Max. group size:             1685              Converged:                    Yes        
Mean group size:             1314.3                                                     
----------------------------------------------------------------------------------------
                                              Coef.  Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------------------------------------
Intercept                                      5.249    0.073 71.738 0.000  5.105  5.392
environment[T.pre_mood]                        

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/statsmodels/base/model.py:1286: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))


In [17]:
r_params = pd.DataFrame(r.params, columns = ["LMM"])
random_effects = pd.DataFrame(r.random_effects)
random_effects = random_effects.transpose()
random_effects = random_effects.rename(index = str, columns = {"group":"LMM"})